In [1]:
import os
import pandas as pd
import pandas_ta as ta
from dotenv import load_dotenv; load_dotenv()

# Load data folder path=
data_path = os.getenv('DATA_PATH')

In [2]:
# Import the data
df = pd.read_parquet(data_path)
df = df[["time", "open", "high", "low", "close", "volume"]]
df = df.set_index("time", drop=True)

df_features = df.copy()
display(df_features.shape)

(3569000, 5)

In [3]:
def calculate_technical_indicators(df):
    # Define individual strategies
    ema_strategies = [{'kind': 'ema', 'length': length} for length in range(2, 100)]
    rsi_strategies = [{'kind': 'rsi', 'length': length} for length in range(2, 100)]
    
    macd_strategies = []
    for fast in range(2, 50):
        for slow in range(10, 100, 5):
            if fast >= slow:
                continue  # Fast period must be less than slow period
            macd_strategies.append({
                'kind': 'macd',
                'fast': fast,
                'slow': slow,
                'signal': 9
            })
    
    bbands_strategies = [{'kind': 'bbands', 'length': length, 'std': 2.0} for length in range(10, 100, 5)]
    
    stoch_strategies = [{'kind': 'stoch', 'k': k, 'd': 3, 'smooth_k': 3} for k in range(5, 100, 5)]
    
    atr_strategies = [{'kind': 'atr', 'length': length} for length in range(2, 100)]
    
    # Combine all strategies
    all_strategies = (ema_strategies + rsi_strategies + macd_strategies +
                      bbands_strategies + stoch_strategies + atr_strategies)
    
    # Create a custom strategy
    custom_strategy = ta.Strategy(
        name="Custom Indicators",
        ta=all_strategies
    )
    
    # Run the strategy
    df.ta.strategy(custom_strategy)
    
    # Prepare a dictionary to hold new columns
    new_columns = {}
    
    # Define a mapping for column renaming
    rename_map = {
        "MACD": lambda parts: f"MACD_{parts[1]}_{parts[2]}_{parts[3]}_{parts[-1]}",
        "BBL": lambda parts: f"Bollinger_{parts[1]}_{parts[2]}_lower",
        "BBM": lambda parts: f"Bollinger_{parts[1]}_{parts[2]}_middle",
        "BBU": lambda parts: f"Bollinger_{parts[1]}_{parts[2]}_upper",
        "STOCHk": lambda parts: f"STOCH_{parts[1]}_{parts[2]}_{parts[3]}_k",
        "STOCHd": lambda parts: f"STOCH_{parts[1]}_{parts[2]}_{parts[3]}_d"
    }
    
    # Rename columns for consistency
    for col in df.columns:
        if col.startswith(("EMA", "RSI", "MACD_", "BBL", "BBM", "BBU", "STOCHk_", "STOCHd_", "ATR")):
            parts = col.split('_')
            indicator = next((k for k in rename_map if col.startswith(k)), None)
            if indicator:
                new_name = rename_map[indicator](parts)
            else:
                new_name = col
            new_columns[new_name] = df[col]
    
    # Create a new DataFrame with the collected columns
    df_features = pd.DataFrame(new_columns, index=df.index)
    
    return df_features

# Usage example
# df_features = calculate_technical_indicators(df)


In [ ]:
# Usage
df_features = calculate_technical_indicators(df)